Looks like another job for `itertools.cycle`...

In [1]:
import itertools as it

I think the way to approach this is to take a slice of the cycle of length $n$, where we are on the $nth$ step, and create a new cycle, starting with $n$. That is:

In [2]:
testStepSize_i=3

In [3]:
# Initial cycle
spinlock_cyc=it.cycle([0])
n=0

# Step forward the given number of steps:
# (actually needs to be testStepSize_i+1, 
# because we need to consume the first element
# before moving on)
for i in range(testStepSize_i+1):
    next(spinlock_cyc)

# Increase n, take an n-sized slice, and create
# a new cycle with n as the first element
n+=1

# I'm getting some weird behaviour, so let's do this
# step-by-step
newCycle_ls=[n]+list(it.islice(spinlock_cyc, 0, n))
print(newCycle_ls)

spinlock_cyc=it.cycle(newCycle_ls)

[1, 0]


Now, if everything's gone according to plan, the first four elements of the cycle should be [1,0,1,0]

In [4]:
assert list(it.islice(spinlock_cyc, 0, 4))==[1,0,1,0]

OK, that's working now.

So for the second cycle, we want the same:

In [5]:
# Step forward the given number of steps:
for i in range(testStepSize_i+1):
    next(spinlock_cyc)

# Increase n, take an n-sized slice, and create
# a new cycle with n as the first element
n+=1

newCycle_ls=[n]+list(it.islice(spinlock_cyc, 0, n))
print(newCycle_ls)

spinlock_cyc=it.cycle(newCycle_ls)

[2, 1, 0]


[2,1,0] is right. A third time should print [3,1,0,2]:

In [6]:
# Step forward the given number of steps:
for i in range(testStepSize_i+1):
    next(spinlock_cyc)

# Increase n, take an n-sized slice, and create
# a new cycle with n as the first element
n+=1

newCycle_ls=[n]+list(it.islice(spinlock_cyc, 0, n))
print(newCycle_ls)

spinlock_cyc=it.cycle(newCycle_ls)

[3, 1, 0, 2]


Good. Let's define a function to return the spinlock after a given number of iterations:

In [7]:
def spinlock_generator(stepSizeIn_i, iterations_i):
    '''
    Return a generator with step size stepSizeIn_i, that
    has run for iterations_i cycles.
    '''

    spinlock_cyc=it.cycle([0])
    for n in range(iterations_i):
        
        # Step forward the given number of steps:
        for i in range(stepSizeIn_i+1):
            next(spinlock_cyc)

        newCycle_ls=[n+1]+list(it.islice(spinlock_cyc, 0, n+1))

        spinlock_cyc=it.cycle(newCycle_ls)
        
    return spinlock_cyc
        

In [8]:
assert it.islice(spinlock_generator(3, 1), 1)

In [9]:
list(it.islice(spinlock_generator(3, 3), 4))

[3, 1, 0, 2]

In [10]:
assert list(it.islice(spinlock_generator(3, 4), 5))==[4,3,1,0,2]
assert list(it.islice(spinlock_generator(3, 5), 6))==[5,2,4,3,1,0]
assert list(it.islice(spinlock_generator(3, 6), 7))==[6,1,0,5,2,4,3]
assert list(it.islice(spinlock_generator(3, 7), 8))==[7,2,4,3,6,1,0,5]

Seems to be working. Now, the task asks for the first element after the inserted element after the 2017th run. Which for the test case will be:

In [11]:
u=spinlock_generator(3, 2017)
next(u)
assert next(u)==638

Good. Now apply to the puzzle input (363):

In [12]:
u=spinlock_generator(363, 2017)
next(u)
next(u)

136